In [1]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from textblob import TextBlob
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

c:\Users\markham\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
test_data = pd.read_csv('data\processed_articles_V8.csv')
test_data['content'] = test_data['title'] + test_data['body']
test_data = test_data.drop(columns=['title','body'])
test_data['date'] = pd.to_datetime(test_data['date'])
test_data = test_data.sort_values(by='date')
test_data['content'] = test_data['content'].apply(lambda x: ' '.join(x.split(',')))
stop_words = set(stopwords.words('english'))
stop_words_list = [ 'china','usa','us','america','american','americans','chinese','china','russia','russian','putin','vladimir','trump','donald','biden','joe','ukraine','ukrainian','ukrainians','ukraines','ukraine','say','jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec','mon','tue','wed','thu','fri','sat','sun','monday','tuesday','wednesday','thursday','friday','saturday','sunday','january','february','march','april','may','june','july','august','september','october','november','december','today','yesterday','tomorrow','week','month','year','time','day','weekend','morning','afternoon','evening','night','news','new','news']
list_numbers = ['one','two','three','four','five','six','seven','eight','nine','ten','hundred','thousand','million','billion','trillion']
stop_words.update(stop_words_list)
stop_words.update(list_numbers)
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word != 'nan']))
test_data['content_tokens'] = test_data['content'].apply(lambda x: x.split())

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\markham\AppData\Local\Temp\ipykernel_23204\176033520.py:1: SyntaxWarning: invalid escape sequence '\p'
  test_data = pd.read_csv('data\processed_articles_V8.csv')


In [3]:
# Define the parameter ranges to search
vector_sizes = [30, 40, 50, 60, 70, 80, 90, 100]
windows = [3, 4, 5, 6, 7, 8, 9, 10]
min_counts = [2, 3, 4, 5, 6, 7]

best_silhouette = -1
best_ch_score = -1
best_params = {}

# Store parameter combinations and their corresponding evaluation metrics
results = []

# Traverse parameter combinations
for vector_size in vector_sizes:
    for window in windows:
        for min_count in min_counts:
            # Train Word2Vec model on important_words (assuming it's a list of tokenized words per row)
            model = Word2Vec(sentences=test_data['content_tokens'].dropna(), vector_size=vector_size, window=window, min_count=min_count, workers=8)

            # Step 3: Define sentiment-related seed words
            positive_seed_words = ["joy", "success", "hope", "love", "peace", "progress", "victory", "happiness", "inspire", "trust"]
            negative_seed_words = ["fear", "failure", "hate", "chaos", "decline", "loss", "anger", "despair", "betrayal", "crisis"]

            # Filter seed words to ensure they are in the model's vocabulary
            positive_seed_words = [word for word in positive_seed_words if word in model.wv]
            negative_seed_words = [word for word in negative_seed_words if word in model.wv]

            if not positive_seed_words or not negative_seed_words:
                print("Warning: Some seed words are not in the model's vocabulary. Check vocabulary or data.")

            # Step 4: Precompute TF-IDF weights for the vocabulary
            corpus = test_data['content'].tolist()  # Use space-separated strings for TF-IDF
            vectorizer = TfidfVectorizer(vocabulary=model.wv.index_to_key)
            tfidf_matrix = vectorizer.fit_transform(corpus)
            word2tfidf = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))

            # Step 5: Precompute similarity to seed words for efficiency
            word_similarities = {}
            for word in model.wv.index_to_key:
                pos_sim = max([model.wv.similarity(word, seed) for seed in positive_seed_words], default=0)
                neg_sim = max([model.wv.similarity(word, seed) for seed in negative_seed_words], default=0)
                word_similarities[word] = max(pos_sim, neg_sim)

            # Step 6: Filter important words (top N most similar to seed words)
            N = 10  # Select top 10 words per article; adjust as needed
            test_data['important_words'] = test_data['content_tokens'].apply(
                lambda tokens: sorted(
                    [token for token in tokens if token in word_similarities],
                    key=lambda x: word_similarities[x],
                    reverse=True
                )[:N]
            )            
            
            # Define a function to convert important_words to a vector
            def text_to_vector(text, model):
                """Convert a list of important words to a weighted average vector."""
                vectors = []
                for word in text:
                    if word in model.wv:
                        vectors.append(model.wv[word])
                if not vectors:
                    # If no valid word vectors, return a zero vector
                    return np.zeros(model.vector_size)
                return np.mean(vectors, axis=0)

            # Convert each row's important_words to a vector
            test_data['vector'] = test_data['important_words'].apply(lambda x: text_to_vector(x, model) if isinstance(x, list) else np.zeros(model.vector_size))
            
            # Extract all news vectors
            X = np.array(test_data['vector'].tolist())

            # Assume number of clusters is 2 (adjust as needed)
            num_clusters = 2

            # Create KMeans model
            kmeans = KMeans(n_clusters=num_clusters, random_state=42)

            # Perform clustering
            labels = kmeans.fit_predict(X)

            # Evaluate clustering effect
            if len(set(labels)) > 1:
                silhouette = silhouette_score(X, labels)
                ch_score = calinski_harabasz_score(X, labels)
                
                # Update best parameters based on silhouette score (primary metric)
                if silhouette > best_silhouette:
                    best_silhouette = silhouette
                    best_ch_score = ch_score
                    best_params = {
                        'vector_size': vector_size,
                        'window': window,
                        'min_count': min_count
                    }
                
                print(f"Parameters: vector_size={vector_size}, window={window}, min_count={min_count}")
                print(f"Silhouette Score: {silhouette:.3f}")
                print(f"Calinski-Harabasz Index: {ch_score:.3f}")
                results.append((vector_size, window, min_count, silhouette, ch_score))
            else:
                print(f"Parameters: vector_size={vector_size}, window={window}, min_count={min_count}, No clustering possible (fewer than 2 clusters).")

# Print the best parameters and scores
print("\nBest Parameter Combination:", best_params)
print("Best Silhouette Score:", best_silhouette)
print("Best Calinski-Harabasz Index:", best_ch_score)

# Optional: Create a DataFrame of results for analysis
results_df = pd.DataFrame(results, columns=['vector_size', 'window', 'min_count', 'silhouette_score', 'calinski_harabasz_score'])
print("\nParameter Search Results:")
print(results_df)

Parameters: vector_size=30, window=3, min_count=2
Silhouette Score: 0.305
Calinski-Harabasz Index: 853.360
Parameters: vector_size=30, window=3, min_count=3
Silhouette Score: 0.416
Calinski-Harabasz Index: 766.726
Parameters: vector_size=30, window=3, min_count=4
Silhouette Score: 0.412
Calinski-Harabasz Index: 751.617
Parameters: vector_size=30, window=3, min_count=5
Silhouette Score: 0.458
Calinski-Harabasz Index: 813.790
Parameters: vector_size=30, window=3, min_count=6
Silhouette Score: 0.339
Calinski-Harabasz Index: 672.677
Parameters: vector_size=30, window=3, min_count=7
Silhouette Score: 0.439
Calinski-Harabasz Index: 879.200
Parameters: vector_size=30, window=4, min_count=2
Silhouette Score: 0.391
Calinski-Harabasz Index: 940.313
Parameters: vector_size=30, window=4, min_count=3
Silhouette Score: 0.426
Calinski-Harabasz Index: 776.913
Parameters: vector_size=30, window=4, min_count=4
Silhouette Score: 0.433
Calinski-Harabasz Index: 827.523
Parameters: vector_size=30, window=4,

In [4]:

max_silhouette_score_row = results_df.loc[results_df['silhouette_score'].idxmax()]
max_calinski_harabasz_score_row = results_df.loc[results_df['calinski_harabasz_score'].idxmax()]

print("Row with the maximum silhouette_score:")
print(max_silhouette_score_row)

print("\nRow with the maximum calinski_harabasz_score:")
print(max_calinski_harabasz_score_row)

Row with the maximum silhouette_score:
vector_size                  50.000000
window                        9.000000
min_count                     4.000000
silhouette_score              0.600883
calinski_harabasz_score    1898.235498
Name: 134, dtype: float64

Row with the maximum calinski_harabasz_score:
vector_size                  70.000000
window                       10.000000
min_count                     7.000000
silhouette_score              0.585516
calinski_harabasz_score    2395.691545
Name: 239, dtype: float64
